# **TP3 - Exercicio 2 - Grupo 11**

Bruno Miguel Fernandes Araújo - A97509

Tiago Emanuel Lemos Teixeira - A97666

Considere-se de novo o algoritmo estendido de Euclides apresentado no TP2  mas usando o tipo dos inteiros e um parâmetro $N>0$

    INPUT  a, b : Int
    assume  a > 0 and b > 0 and a < N and b < N
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
    OUTPUT r, s, t


## Exercício 2

Este exercício é dirigido à prova de correção do algoritmo estendido de Euclides.

   a. Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função  $\gcd$  é   $$\gcd(a,b)\;\equiv\; \min \{\,r > 0\,|\,\exists\,s,t\,\centerdot\, r = a*s+b*t\,\}$$.
   
   b. Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .
   
   c. Construa codificações do programa LPA através de transformadores de predicados: “weakest pre-condition” e “strongest post-condition”. 
   
   d. Prove a correção  do programa LPA em ambas as codificações.

# Resolução das alíneas

### 2 A)

Tendo em conta a expressão denotada abaixo sobre a função $\gcd(a,b)$, 

 $$\gcd(a,b)\;\equiv\; \min \{\,r > 0\,|\,\exists\,s,t\,\centerdot\, r = a*s+b*t\,\}$$ 
 

A seguinte expressão representa a pós-condição do algoritmo em questão, constituída por:

$\text{r}$ - Resultado de $\gcd(a,b)$ (máximo divisor comum entre a e b), com $\text{s,t} \in \text{Z}$; $\\$
$\text{N}$ - Valor a que $\text{r}$ está limitado superiormente; $$\\$$
$\text{rNew}$ - valor menor que $\text{r}$ para algum $\text{sNew,tNew} \in \text{Z}$



$$pos =$$ $$ \text{i.}( 0 < r < N \wedge r == a * s + b * t)$$ $$\wedge$$ $$ \text{ii.} \neg ( 0 < rNew < r \wedge rNew == a * sNew + b * tNew)$$


## 2 B)

### LPA com metodologia de comando havoc

Tradução para um programa de fluxos:

Sejam $(i)$: 
```Python

    pre = a > 0 and b > 0 and a < N and b < N
    pos = ( 0 < r < N and r == a * s + b * t) and not( 0 < rNew < r and rNew == a * sNew + b * tNew)
    inv = 0 < r < N and r == a * s + b * t
```


Sendo $\text{pre}$ e $\text{inv}$ as asserções lógicas, que representam respetivamente a "pré-condição" e o "invariante" do pseudo-código apresentado no enunciado deste trabalho.

Em baixo mostra-se o código traduzido em função da Linguagem de Programas Anotados (LPA)

```Python
assume pre;
r, r_, s, s_, t, t_ = a, b, 1, 0, 0, 1;
(assert inv;
havoc r; havoc s; havoc t; havoc r_; havoc s_; havoc t_;
#while r_ != 0
[( assume r_ != 0 and inv;
   q = r div r_;
   r, r_, s, s_, t, t_ = r_, r − q * r_, s_, s − q * s_, t_, t − q * t_;
   assert inv; assume False;)
|| 
   assume not(r_ != 0) and inv;])

assert pos;
```


Mostramos, já agora, o código em python referente às condições de $(i)$ (a,b e N são valores escolhidos pelo utilizador, onde $\text{0}$ < $\text{a,b}$ < $\text{N}$ ), que será preciso brevemente

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *
N=50
a = 42
b = 28

r = Symbol('r',INT)
s = Symbol('s',INT)
t = Symbol('t',INT)


pre = And(GT(Int(a),Int(0)),GT(Int(b),Int(0)),LT(Int(a),Int(N)),LT(Int(b),Int(N)))

def inv(r,a,b,s,t):
    return And(GT(r,Int(0)),LT(r,Int(N)),Equals(r, (Int(a) * s) + (Int(b) * t)))

def R_isDivisor(a,b,r):
    newR = Symbol(f'newR',INT)
    newS = Symbol(f'newS',INT)
    newT = Symbol(f'newT',INT)
    
    return And(GT(newR,Int(0)),LT(newR, r), inv(newR,a,b,newS,newT))

pos = And(Not(R_isDivisor(a,b,r)),inv(r,a,b,s,t))



$Nota:$ a função definida por $\text{R_isDivisor(a,b,r)}$, refere-se à afirmação negada, que se encontra à direita da conjunção da pós-condição (Rever exercício $\text{2a)ii.}$) 

## 2 C)

### Denotação lógica com WPC

Utilizando como guias, as fórmulas que foram dadas na resolução da Ficha 7.
Tais fórmulas transformam excertos de código LPA para afirmações lógicas

Weakest Pre-condition

$
\begin{array}{l}
1) [{\sf skip}] = True \\
2) [{\sf assume}\:\phi] = True \\
3)[{\sf assert}\:\phi] = \phi \\
4)[ x = e ] = True \\
5)[(C_1 || C_2)] = [C_1] \wedge [C_2] \\
\\
6)[{\sf skip}\, ; C] = [C] \\
7)[{\sf assume}\:\phi\, ; C] = \phi \to [C] \\
8)[{\sf assert}\:\phi\, ; C] = \phi \wedge [C] \\
9)[ x = e \, ; C] = [C][e/x] \\
10)[(C_1 || C_2)\, ; C] = [(C_1 ; C) || (C_2 ; C)] \\
11)[{\sf havoc}\; x \; ; C] = \forall x. \,[C]
\end{array}
$

#### Processo de transformação em passos

```Python
[assume pre;
r, r_, s, s_, t, t_ = a, b, 1, 0, 0, 1;

assert inv;
havoc r; havoc s; havoc t; havoc r_; havoc s_; havoc t_;

[( assume r_ != 0 and inv;
   q = r div r_;
   r, r_, s, s_, t, t_ = r_, r − q × r_, s_, s − q × s_, t_, t − q × t_;
   assert inv; assume False;)
|| 
   assume not(r_ != 0) and inv;]
assert pos;]

= # 7) e 9)
pre -> [assert inv; havoc r; havoc s; havoc t; havoc r_; havoc s_; havoc t_; ...; assert pos;] [1/t_][0/t][0/s_][1/s][b/r_][a/r]

= # 8) e 11)
pre -> inv[1/t_][0/t][0/s_][1/s][b/r_][a/r] and 
       (forall r,s,t,r_,s_,t_ . [ ...; assert pos;]) [1/t_][0/t][0/s_][1/s][b/r_][a/r]
    
= # 7), 9), 5), 3) 
pre -> inv[1/t_][0/t][0/s_][1/s][b/r_][a/r] and  # init
       (forall r,s,t,r_,s_,t_ .                                                                       # 
           ((r_ != 0 and inv) ->                                                                      # pres
             inv[[(t - q × t_)/t_][t_/t][(s − q × s_)/s_][s_/s][(r − q × r_)/r_][r_/r]][(r div r_)/q] #
           )                                          
                                        and
                           ((not(r_ != 0) and inv)) -> pos  # util
       
       
```

Código em python que representa as asserções lógicas que servirão para provar a veracidade do invariante, $\text{init_WPC}$ (inicialização), $\text{pres_WPC}$ (preservação) e $\text{util_WPC}$ (utilidade) obtidas pelo processo de transformação:

In [3]:
r_ = Symbol('r_',INT)
s_ = Symbol('s_',INT)
t_ = Symbol('t_',INT)
q =  Symbol('q',INT)

init_WPC = substitute(inv(r,a,b,s,t),{t_:Int(1), t:Int(0), s_:Int(0), s:Int(1), r_:Int(b), r:Int(a)})
pres_WPC = Implies(And(Not(Equals(r_,Int(0))), inv(r,a,b,s,t)),
                                    substitute(
                                        substitute(
                                           inv(r,a,b,s,t),
                                           {t_:(t - (q * t_)),t:t_,
                                            s_:(s - (q * s_)),s:s_,
                                            r_:(r - (q * r_)),r:r_}
                                                  ),
                                        {q:Div(r, r_)})
                  )
util_WPC = Implies(And(Equals(r_,Int(0)),inv(r,a,b,s,t)),pos)


A seguinte função "prove(f)", retirada da Ficha 7, vai servir para provar a correçao do programa em LPA

In [4]:
def prove(f):
    with Solver(name="z3") as s:
        s.add_assertion(Not(f))
        if s.solve():
            print("Failed to prove.")
        else:
            print("Proved.")

O próximo excerto de código em python, vai ser usado para provar a tal correção sendo "$\text{vc_WPC}$" a variável que representa o programa todo

In [4]:
gcd = Symbol('gcd', FunctionType(INT,[INT,INT]))
Sa = Symbol('Sa',INT)
Sb = Symbol('Sb',INT)

ax1 = Equals(gcd(Sa,Int(0)),Sa)
ax3 = ForAll([Sa,Sb], Implies(GT(Sa,Sb),Ite(LT(Sa-Sb,Sb),Equals(gcd(Sa,Sb),gcd(Sb,Sa-Sb)),Equals(gcd(Sa,Sb),Sb))))
axioms = And(ax1,ax3)

vc_WPC = Implies(pre , And(init_WPC,ForAll([r,r_,s,s_,t,t_],And(pres_WPC,util_WPC))))

prove(Implies(pre,init_WPC))
prove(ForAll([r,r_,s,s_,t,t_,q],pres_WPC))
prove(vc_WPC)

prove(Implies(axioms,Equals(gcd(Int(a),Int(b)),Int(14))))

Proved.
Failed to prove.
Failed to prove.
Proved.


### Denotação lógica com SPC

Apartir daqui o trabalho está imcompleto, e por sua vez não dá pra provar a correção do código em denotação lógica usando a técnica SPC

Para que a traduçao de programa de fluxos para LPA faça sentido, tivemos que mudar ligeiramente o código original.

##### Mudança de nome de variáveis:

```Python
assume pre;
r, r_, s, s_, t, t_ = a, b, 1, 0, 0, 1;
rdic = {}; r_dic = {}; sdic = {}; s_dic = {}; tdic = {}; t_dic = {};
rdic[0] = r;
r_dic[0] = r_;
sdic[0] = s;
s_dic[0] = s_;
tdic[0] = t;
t_dic[0] = t_;
i = 0;

assert inv;
[havoc i; # havoc r; havoc s; havoc t; havoc r_; havoc s_; havoc t_;

( assume r_dic[i] != 0 and inv; # init
  
   q = rdic[i] div r_dic[i];                                  #                                                    
   rdic[i+1] = r_dic[i]; r_dic[i+1] = rdic[i] − q × r_dic[i]; #
   sdic[i+1] = s_dic[i]; s_dic[i+1] = sdic[i] − q × s_dic[i]; # 
   tdic[i+1] = t_dic[i]; t_dic[i+1] = tdic[i] − q × t_dic[i]; # pres
                                                              #
   assert inv; assume False;)                                 #
 
|| 
   assume not(r_dic[i] != 0) and inv;]                         #
                                                               # util
assert pos;                                                    #
```

##### Novas variáveis usadas

"--dic" é um dicionário de variáveis lógicas (keys:nº identificador da variável;values: o que representa a variável), que estará constantemente a crescer, consoante a necessidade de criar variáveis novas, onde "--" é:

- $\text{r}$
- $\text{r'}$
- $\text{s}$
- $\text{s'}$
- $\text{t}$
- $\text{t'}$

Strongest Post-Codition


$
\begin{array}{l}
1) [{\sf skip}] = True \\
2) [{\sf assume}\:\phi] = \phi \\
3) [{\sf assert}\:\phi] = \phi \\
4) [x = e ] = (x = e)\\
5) [(C_1 || C_2)] = [C_1] \vee [C_2] \\
\\
6) [C \, ; {\sf skip}\;] = [C] \\
7) [C \, ;{\sf assume}\:\phi] = [C] \wedge \phi \\
8) [C \, ;{\sf assert}\:\phi] = [C] \to \phi \\
9) [ C \, ; x = e ] = [C] \wedge (x = e)\\
10) [C\,; (C_1 || C_2)] = [(C ; C_1) || (C; C_2)] \\
11) [{\sf havoc}\; x \; ; C] = \forall x. \,[C]
\end{array}
$

#### Processo de transformação em passos

```Python
[assume pre;
r, r_, s, s_, t, t_ = a, b, 1, 0, 0, 1;
rdic = {}; r_dic = {}; sdic = {}; s_dic = {}; tdic = {}; t_dic = {}
rdic[0] = r;
r_dic[0] = r_;
sdic[0] = s;
s_dic[0] = s_;
tdic[0] = t;
t_dic[0] = t_;
i = 0;

(assert inv;
havoc i; # havoc r_; havoc s_; havoc t_;

[( assume r_dic[i] != 0 and inv;                               
  
   q = rdic[i] div r_dic[i];                                                                                       
   rdic[i+1] = r_dic[i]; r_dic[i+1] = rdic[i] − q × r_dic[i]; 
   sdic[i+1] = s_dic[i]; s_dic[i+1] = sdic[i] − q × s_dic[i]; 
   tdic[i+1] = t_dic[i]; t_dic[i+1] = tdic[i] − q × t_dic[i]; 
                                                    
   assert inv; assume False;)                                  
 
|| 
   assume not(r_dic[i] != 0) and inv;])                         
                                                               
assert pos;]
#################################################################################################

=      

([assume pre;
r, r_, s, s_, t, t_ = a, b, 1, 0, 0, 1;
rdic = {}; r_dic = {}; sdic = {}; s_dic = {}; tdic = {}; t_dic = {}
rdic[0] = r;
r_dic[0] = r_;
sdic[0] = s;
s_dic[0] = s_;
tdic[0] = t;
t_dic[0] = t_;
i = 0;
  
(assert inv;
havoc i; # havoc r_; havoc s_; havoc t_;

( assume r_dic[i] != 0 and inv;                               
  
   q = rdic[i] div r_dic[i];                                                                                       
   rdic[i+1] = r_dic[i]; r_dic[i+1] = rdic[i] − q × r_dic[i]; 
   sdic[i+1] = s_dic[i]; s_dic[i+1] = sdic[i] − q × s_dic[i]; 
   tdic[i+1] = t_dic[i]; t_dic[i+1] = tdic[i] − q × t_dic[i];
 
   assert inv; assume False;))                                  
 
]|| 
   [... ; assume not(r_dic[i] != 0) and inv;]
)
-> pos # 8) e 10)

#################################################################################################

=

[pre and                                                                                     # 2
r == a and r_ == b and s == 1 and s_ == 0 and t == 0 and t_ == 1 and                         # 9 sucessivos
  
rdic == {} and r_dic == {} and sdic == {} and s_dic == {} and tdic == {} and t_dic == {} and # 9 sucessivos
  
rdic[0] == r and         #
r_dic[0] == r_ and       #
sdic[0] == s and         #
s_dic[0] == s_ and       # 9 sucessivos
tdic[0] == t and         #
t_dic[0] == t_ and       #
i == 0                   #

-> inv;                  # 8
  
havoc i; 

( assume r_dic[i] != 0 and inv;                               
  
   q = rdic[i] div r_dic[i];                                                                                       
   rdic[i+1] = r_dic[i]; r_dic[i+1] = rdic[i] − q × r_dic[i]; 
   sdic[i+1] = s_dic[i]; s_dic[i+1] = sdic[i] − q × s_dic[i]; 
   tdic[i+1] = t_dic[i]; t_dic[i+1] = tdic[i] − q × t_dic[i];
 
   assert inv; assume False;)                                  
 
]|| 
    [pre and                                                                                      # 2
     r == a and r_ == b and s == 1 and s_ == 0 and t == 0 and t_ == 1 and                         # 9 sucessivos
  
     rdic == {} and r_dic == {} and sdic == {} and s_dic == {} and tdic == {} and t_dic == {} and # 9 sucessivos
  
     rdic[0] == r and         #
     r_dic[0] == r_ and       #
     sdic[0] == s and         #
     s_dic[0] == s_ and       # 9 sucessivos
     tdic[0] == t and         #
     t_dic[0] == t_ and       #
     i == 0                   #

     -> inv;                  # 8
     
     havoc i; not(r_dic[i] != 0) and inv;]  

-> pos

#############################################################################################

=

([pre and                                                                                     
r == a and r_ == b and s == 1 and s_ == 0 and t == 0 and t_ == 1 and                         
  
rdic == {} and r_dic == {} and sdic == {} and s_dic == {} and tdic == {} and t_dic == {} and 
  
rdic[0] == r and         
r_dic[0] == r_ and       
sdic[0] == s and         
s_dic[0] == s_ and       
tdic[0] == t and         
t_dic[0] == t_ and       
i == 0                   

-> inv and                     # 9                
  
forall i.                      # 11                                                #
                                                                                   #
(         r_dic[i] != 0 and inv and                                          # 2   #                                  
                                                                                   # Tudo isto   
          q == rdic[i] div r_dic[i] and                                      #     #                                     
          rdic[i+1] == r_dic[i] and r_dic[i+1] == rdic[i] − q × r_dic[i] and #     # é igual a
          sdic[i+1] == s_dic[i] and s_dic[i+1] == sdic[i] − q × s_dic[i] and # 9   #
          tdic[i+1] == t_dic[i] and t_dic[i+1] == tdic[i] − q × t_dic[i]    #      # False por causa                          
          -> inv        # 8                                                        # 
             and False) # 7                                                        # <- desta linha
 
] or  # 5
     [pre and                                                                                      
             r == a and r_ == b and s == 1 and s_ == 0 and t == 0 and t_ == 1 and                         
  
             rdic == {} and r_dic == {} and sdic == {} and s_dic == {} and tdic == {} and t_dic == {} and 
  
             rdic[0] == r and         
             r_dic[0] == r_ and       
             sdic[0] == s and         
             s_dic[0] == s_ and       
             tdic[0] == t and         
             t_dic[0] == t_ and       
             i == 0                   

     -> inv and  # 9                                            
     
                 forall i. not(r_dic[i] != 0) and inv]  # 11 e 7

-> pos)
```

Código em python que representa as asserções lógicas que servirão para provar a veracidade do invariante, $\text{Before_while}$ (inicialização), $\text{C1}$ (preservação) e $\text{C2}$ (utilidade) obtidas pelo processo de transformação (imcompleto), e uma tentativa de provar a correção do programa em denotaçao SPC:

In [6]:
r = Symbol('r',INT)
s = Symbol('s',INT)
t = Symbol('t',INT)
r_ = Symbol('r_',INT)
s_ = Symbol('s_',INT)
t_ = Symbol('t_',INT)
q =  Symbol('q',INT)
i =  Symbol('i',INT)

rdic = {}
r_dic = {}
sdic = {}
s_dic = {}
tdic = {}
t_dic = {}
qdic = {}

def declare_var(string,dict_,i):
    dict_[i] = Symbol(f'{string}'+str(i),INT)
    return dict_[i]

rdic[Int(0)] = declare_var("r",rdic,0)          
r_dic[Int(0)] = declare_var("r_",rdic,0)      
sdic[Int(0)] = declare_var("s",rdic,0)         
s_dic[Int(0)] = declare_var("s_",rdic,0)       
tdic[Int(0)] = declare_var("t",rdic,0)         
t_dic[Int(0)] = declare_var("t_",rdic,0)
#qdic[Int(0)] = declare_var("q",rdic,0)

Before_while = Implies(And(pre,Equals(r,Int(a)),Equals(r_,Int(b)),Equals(s,Int(1)),Equals(s_,Int(0)),Equals(t,Int(0)),Equals(t_,Int(1))
                          ,Equals(rdic[0],r),Equals(sdic[0],s),Equals(tdic[0],t)
                          ,Equals(r_dic[0],r_),Equals(s_dic[0],s_),Equals(t_dic[0],t_),Equals(i,Int(0)))
                   ,inv)


C1 = And(Before_while, ForAll([i],Implies(And(Not(Equals(r_dic[i],Int(0))),inv,Equals(declare_var("q",qdic,i),Div(rdic[i],r_dic[i])),
                        Equals(declare_var("r",rdic,i+Int(1)),r_dic[i]), Equals(declare_var("r_",r_dic,i+Int(1)),rdic[i] - (qdic[i] * r_dic[i])),
                        Equals(declare_var("s",sdic,i+Int(1)),s_dic[i]), Equals(declare_var("s_",s_dic,i+Int(1)),sdic[i] - (qdic[i] * s_dic[i])),
                        Equals(declare_var("t",tdic,i+Int(1)),t_dic[i]), Equals(declare_var("t_",t_dic,i+Int(1)),tdic[i] - (qdic[i] * t_dic[i])))
                                         ,And(inv,False))))

C2 = And(Before_while, ForAll([i],And(Equals(r_dic[Int(len(r_dic)-Int(1))],Int(0)),inv)))

vc_SPC = (Or(C1,C2),pos)

prove(vc_SPC)

AttributeError: 'int' object has no attribute 'node_type'